In [1]:
# 📒 02_build_features_fastf1.ipynb
# Enrich 2025 race data with qualifying times, weather, and optional telemetry features

import fastf1
import pandas as pd
from tqdm import tqdm
from fastf1.core import Laps
from datetime import datetime

# Enable FastF1 cache
fastf1.Cache.enable_cache("../cache")

# Load base dataset
df = pd.read_csv("../data/f1_features_2025_fastf1.csv")
df['date'] = pd.to_datetime(df['date'])

# Extract unique rounds to process
rounds = df['round'].unique()
season = 2024

# New feature containers
qualifying_times = []
weather_data = []
telemetry_data = []

for rnd in tqdm(rounds, desc="Processing Rounds"):
    try:
        session_q = fastf1.get_session(season, rnd, 'Q')
        session_q.load()

        # 1. Build full name lookup from session results
        full_name_lookup = session_q.results.set_index('Abbreviation')['FullName'].to_dict()

        # 2. Get best lap time per driver
        laps_q = session_q.laps.pick_quicklaps()
        best_laps = laps_q.groupby('Driver')['LapTime'].min().reset_index()
        print(f"🔎 Best laps (Driver column) for round {rnd}:")
        print(best_laps['Driver'].unique())

        best_laps['qualifying_time'] = best_laps['LapTime'].dt.total_seconds()

        # 3. Map abbreviation to full name
        best_laps['driver'] = best_laps['Driver'].map(full_name_lookup)
        best_laps = best_laps.dropna(subset=['driver'])  # safety check

        # 4. Finalize and store
        best_laps['round'] = rnd
        best_laps = best_laps[['driver', 'qualifying_time', 'round']]
        qualifying_times.append(best_laps)

        # 5. Weather data
        weather = session_q.weather_data.iloc[0]
        weather_data.append({
            'round': rnd,
            'air_temp': weather['AirTemp'],
            'track_temp': weather['TrackTemp'],
            'humidity': weather['Humidity']
        })

        # 6. Optional telemetry
        try:
            telemetry_avg = laps_q.groupby("Driver").agg({
                'SpeedST': 'mean',
                'Throttle': 'mean',
                'Brake': 'mean'
            }).reset_index()
            telemetry_avg['round'] = rnd
            telemetry_data.append(telemetry_avg)
        except KeyError:
            print(f"⚠️ Telemetry missing for round {rnd} — skipping telemetry columns.")

    except Exception as e:
        print(f"⚠️ Qualifying data missing for round {rnd}: {e}")
        continue

# Merge all data
if qualifying_times:
    df_q = pd.concat(qualifying_times, ignore_index=True)
    df_enriched = df.merge(df_q, on=['round', 'driver'], how='left')
else:
    print("❌ No qualifying data loaded. Exiting.")
    raise SystemExit

df_weather = pd.DataFrame(weather_data)
df_enriched = df_enriched.merge(df_weather, on='round', how='left')

if telemetry_data:
    df_tele = pd.concat(telemetry_data, ignore_index=True)
    df_enriched = df_enriched.merge(df_tele, left_on=['round', 'driver'], right_on=['round', 'Driver'], how='left')
    df_enriched.drop(['Driver'], axis=1, inplace=True)
else:
    print("⚠️ No telemetry data found — skipping telemetry merge.")

# Save final enriched dataset
df_enriched.to_csv("../data/f1_features_2025_enriched.csv", index=False)
print("✅ Enriched dataset saved as 'f1_features_2024_enriched.csv'")
df_enriched.head()


Processing Rounds:   0%|                                                                        | 0/22 [00:00<?, ?it/s]core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '55', '11', '14', '4', '81', '44', '27', '22', '18', '23', '3', '20', '77', '24', '2', '3

🔎 Best laps (Driver column) for round 1:
['ALB' 'ALO' 'BOT' 'GAS' 'HAM' 'HUL' 'LEC' 'MAG' 'NOR' 'OCO' 'PER' 'PIA'
 'RIC' 'RUS' 'SAI' 'SAR' 'STR' 'TSU' 'VER' 'ZHO']
⚠️ Telemetry missing for round 1 — skipping telemetry columns.


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '14', '81', '4', '63', '44', '22', '18', '38', '23', '20', '3', '27', '77', '31', '10', '2', '24']
Processing Rounds:   9%|█████▊                                                          | 2/22 [00:03<00:38,  1.93s/it]core           INFO 	Loading data for Australian Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for

🔎 Best laps (Driver column) for round 2:
['ALB' 'ALO' 'BEA' 'BOT' 'GAS' 'HAM' 'HUL' 'LEC' 'MAG' 'NOR' 'OCO' 'PER'
 'PIA' 'RIC' 'RUS' 'SAR' 'STR' 'TSU' 'VER']
⚠️ Telemetry missing for round 2 — skipping telemetry columns.


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 19 drivers: ['1', '55', '11', '4', '16', '81', '63', '22', '18', '14', '44', '23', '77', '20', '31', '27', '10', '3', '24']
Processing Rounds:  14%|████████▋                                                       | 3/22 [00:05<00:34,  1.81s/it]core           INFO 	Loading data for Japanese Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver

...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['4', '63', '22', '31', '30', '16', '23', '81', '14', '18', '77', '1', '11', '55', '10', '44', '50', '43', '27', '24']
Processing Rounds:  95%|████████████████████████████████████████████████████████████▏  | 21/22 [00:39<00:02,  2.30s/it]core           INFO 	Loading data for Las Vegas Grand Prix - Qualifying [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data fo

🔎 Best laps (Driver column) for round 21:
['ALB' 'ALO' 'BOT' 'LAW' 'LEC' 'NOR' 'OCO' 'PER' 'PIA' 'RUS' 'STR' 'TSU'
 'VER']
⚠️ Telemetry missing for round 21 — skipping telemetry columns.


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '55', '10', '16', '1', '4', '22', '81', '27', '44', '31', '20', '24', '43', '30', '11', '14', '23', '77', '18']
Processing Rounds: 100%|███████████████████████████████████████████████████████████████| 22/22 [00:41<00:00,  1.90s/it]

🔎 Best laps (Driver column) for round 22:
['ALB' 'ALO' 'BOT' 'COL' 'GAS' 'HAM' 'HUL' 'LAW' 'LEC' 'MAG' 'NOR' 'OCO'
 'PER' 'PIA' 'RUS' 'SAI' 'STR' 'TSU' 'VER' 'ZHO']
⚠️ Telemetry missing for round 22 — skipping telemetry columns.
⚠️ No telemetry data found — skipping telemetry merge.
✅ Enriched dataset saved as 'f1_features_2024_enriched.csv'


,date,season,round,circuit,driver,constructor,grid,position,qualifying_time,air_temp,track_temp,humidity
0,2024-03-02,2024,1,Bahrain Grand Prix,Max Verstappen,Red Bull Racing,1.0,1.0,89.179,18.2,22.2,48.0
1,2024-03-02,2024,1,Bahrain Grand Prix,Sergio Perez,Red Bull Racing,5.0,2.0,89.537,18.2,22.2,48.0
2,2024-03-02,2024,1,Bahrain Grand Prix,Carlos Sainz,Ferrari,4.0,3.0,89.507,18.2,22.2,48.0
3,2024-03-02,2024,1,Bahrain Grand Prix,Charles Leclerc,Ferrari,2.0,4.0,89.165,18.2,22.2,48.0
4,2024-03-02,2024,1,Bahrain Grand Prix,George Russell,Mercedes,3.0,5.0,89.485,18.2,22.2,48.0
